In [0]:
%sql
SELECT
    HOUR(pickup_datetime) AS hour,
    COUNT(*) AS trip_count
FROM
    (SELECT pickup_datetime FROM yellow_trips_01
     UNION ALL
     SELECT pickup_datetime FROM yellow_trips_02
     UNION ALL
     SELECT pickup_datetime FROM yellow_trips_03)
GROUP BY
    hour
ORDER BY
    trip_count DESC;

In [0]:
%sql
SELECT
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    COUNT(*) AS trip_count
FROM
    trip_data
GROUP BY
    hour
ORDER BY
    hour;


In [0]:
%sql
SELECT
    passenger_count,
    AVG(fare_amount) AS average_fare
FROM
    taxi_trip_data
GROUP BY
    passenger_count
ORDER BY
    passenger_count;


In [0]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def create_visualizations(db_name):
    conn = sqlite3.connect(db_name)
    
    # Peak hours visualization
    query = "SELECT strftime('%H', pickup_datetime) as hour, COUNT(*) as total_trips FROM trips GROUP BY hour ORDER BY total_trips DESC LIMIT 10;"
    peak_hours = pd.read_sql(query, conn)
    sns.barplot(x='hour', y='total_trips', data=peak_hours)
    plt.title("Peak Hours for Taxi Usage")
    plt.show()
    
    # Passenger count effect on fare
    query = "SELECT passenger_count, AVG(fare_amount) as average_fare FROM trips GROUP BY passenger_count;"
    passenger_fares = pd.read_sql(query, conn)
    sns.barplot(x='passenger_count', y='average_fare', data=passenger_fares)
    plt.title("Effect of Passenger Count on Fare")
    plt.show()
    
    # Trends over the year
    query = "SELECT date, total_trips, average_fare FROM daily_aggregates ORDER BY date;"
    trends = pd.read_sql(query, conn)
    trends['date'] = pd.to_datetime(trends['date'])
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(trends['date'], trends['total_trips'], 'g-')
    ax2.plot(trends['date'], trends['average_fare'], 'b-')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Total Trips', color='g')
    ax2.set_ylabel('Average Fare', color='b')
    plt.title("Trends in Taxi Usage Over the Year")
    plt.show()
    
    conn.close()

if __name__ == "__main__":
    create_visualizations("ny_taxi.db")
